# Generate a mosaic of the GOES-16 ABI channels with Satpy and ImageMagick

This example shows the basic functionality of reading GOES-16 ABI data and creating a PNG grayscale image for each of the 16 channels. The `montage` command line utility from `ImageMagick` is then used to join the individual images in to one image. In this first cell we do all of this using data from Amazon Web Services (AWS) from the S3 bucket created by NOAA.

We use the feature of the `fsspec` library called `simplecache` to store files locally. This makes sure that any repeated access to the same files will be fast and use the local cache. In the below code, accessing the remote data the first time and processing it with Satpy will take about 4 minutes depending on your network connection and your local system's resources. Running the same cell again will use the cache and take around 1 minute and 45 seconds.

In [ ]:
%%time

import glob
from satpy import Scene
from satpy.readers import FSFile
import fsspec

# Access data from AWS S3 to be read by Satpy
filename = 'noaa-goes16/ABI-L1b-RadF/2019/001/17/*_G16_s201900117003*'
the_files = fsspec.open_files("simplecache::s3://" + filename, s3={'anon': True})
fs_files = [FSFile(open_file) for open_file in the_files]

# Pass S3 file objects to Satpy and read them with the 'abi_l1b' reader
scn = Scene(filenames=fs_files, reader='abi_l1b')

# Load all 16 ABI channels
scn.load(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08',
          'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'])

# Save each channel to a PNG image
scn.save_datasets(filename='{name}.png')

Then we can use the ImageMagick command `montage` to join all the images together.

Note: Some ImageMagick installations may be limited and unable to process every image in the montage.

In [ ]:
!montage C??.png -tile 4x4 -geometry 128x128+4+4 -background black montage_abi.png

In [ ]:
from IPython.display import Image
Image(filename='montage_abi.png') 

# Local Files

Alternatively, if you have ABI L1b files on your local machine already, you can point Satpy to those directly:

In [ ]:
import glob
from satpy.scene import Scene

# Access local ABI L1b NetCDF files
scn = Scene(
    filenames=glob.glob("/path/to/the/GOES-16/data/*RadF*.nc"),
    reader='abi_l1b'
)

# Load all 16 ABI channels
scn.load(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08',
          'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'])

# Save each channel to a PNG image
scn.save_datasets(filename='{name}.png')

# Additional Information

1. To avoid S3 files being cached on your local system, modify the `fsspec.open_files` line to say:

   ```
   the_files = fsspec.open_files("s3://" + filename, anon=True)
   ```
   
   Note however that this means that any repeated access to file content will take longer and will increase the overall processing time.
   
2. To save images in the GeoTIFF format, you can call `scn.save_datasets()` without any arguments as geotiffs are the default.